# Weekly Reporting
This code produces an Excel file with weekly reporting data.

### Directions:
1. Ensure Rollforward.csv is in the proper location and updated using Galileo data
2. Update 'Reusable Stuff.ipynb' if necessary
3. Update FILE_PATH if necessary

### To Do:
1. Merge data with transaction table
2. Improve way to isolate incentive checks (based off response from FIS???)
3. Add charts

## Import reusable variables and functions

In [1]:
%run 'Reusable Stuff.ipynb'

## Define variables for analysis
Variables include # of weeks in a pay period, first day of first pay period, last day in dataset, accounts to exclude from analysis, and current salespeople.

In [2]:
# directory path for where to save file
file_path = '/users/kevinkupiec/Dropbox (One Financial)/OFH Share/Reporting & Analysis/Reporting/Weekly Updates/'


# number of weeks per period
period_length = 1       


# number of weeks per cohort
cohort_length = 2   


# rolling days in analysis
rolling_days = 45


# day of start of first pay period
first_day = Timestamp('2015-06-22')


# define last day for analysis
last_day = rollforward2_last_day


# create dict for PRN mapping
with open(prn_mapping_file_name) as f:
    f.readline() # ignore first line (header)
    prn_mapping = dict(csv.reader(f, delimiter=','))

## Define functions to help complete analysis

## Create initial data tables from database

Note that values in table **df_txn** currently come from Galileo data as transaction table is not yet complete.

In [3]:
# connect to database
db = connect()

In [4]:
# input account data including signup date
#     note: shift from TNCs = True to Galileo Enrollment = Confirmed added one signup 
#     in between June 29 & July 5, 2015; i do not believe it's a test account
df_acct_input = get_dataframe(
"""
SELECT
    a.account_owner_uid,
    a.ach_account_number as pmt_ref_no,
    u.created_at AS signup_date,
    u.roles AS account_roles
FROM accounts AS a 
INNER JOIN users AS u 
    ON u.uid = a.account_owner_uid
LEFT OUTER JOIN partner_enrollments AS pe
    ON a.galileo_account_enrollment_uid = pe.uid  
WHERE a.product_uid = '00fe52dda6c028c44db8485296d6855eacc'
    AND pe.status = 'confirmed'
ORDER BY signup_date DESC
"""
) 

In [5]:
# input transaction data from Galileo - this is all posted data & date is date posted
df_txn_input = pd.read_csv(rollforward2_file_name, parse_dates=[3])
df_txn_input['pmt_ref_no'] = df_txn_input['pmt_ref_no'].astype(str)


# map federal benefits PRNs
df_txn_input['pmt_ref_no'] = df_txn_input['pmt_ref_no'].apply(lambda x: prn_mapping[x] if x in prn_mapping else x)


# remove transactions from products we don't care about
df_txn_input = df_txn_input[df_txn_input['prod_id'].isin((5105, 5106, 5197)) == True]

## Create working copies of tables

In [6]:
df_acct = df_acct_input.copy()
df_txn = df_txn_input.copy()

## Clean data & add new data fields to create working transaction and account tables

In [7]:
# create field with transaction type
txn_type_args = (txn_type, 'activity_type', 'activity_subtype', 'amount')
df_txn['transaction_type'] = df_txn.apply(append_derived_column, axis=1, args=txn_type_args)


# apply check classification function to create new column; incentive check considered any check that is $1, $9.90, or $10
check_type_args = (incentive_check, 'amount', 'activity_type', 'activity_subtype')
df_txn['incentive_check'] = df_txn.apply(append_derived_column, axis=1, args=check_type_args)
df_txn['transaction_type'] = df_txn.apply(lambda row: 'Incentive Check' if row['incentive_check'] == True else row['transaction_type'], axis=1)


# add account_owner_uid & clean transaction table (will make transition to database from Galileo easier to integrate)
df_txn = df_txn.merge(df_acct, on='pmt_ref_no', how='inner')
df_txn = df_txn[['account_owner_uid', 'account_roles', 'row', 'posted_date', 'transaction_type', 'amount']]
df_txn.columns = ['account_owner_uid', 'account_roles', 'transaction_uid', 'posted_at', 'transaction_type', 'amount']


# append field that identifies loads
load_args = (load, 'transaction_type', 'amount')
df_txn['load_ind'] = df_txn.apply(append_derived_column, axis=1, args=load_args)


# change check load transaction types to 'Check Deposit'
df_txn['transaction_type'] = df_txn.apply(lambda row: 'Check Deposit' if row['transaction_type'] == 'Check Deposit (Instant)' else row['transaction_type'], axis=1)
df_txn['transaction_type'] = df_txn.apply(lambda row: 'Check Deposit' if row['transaction_type'] == 'Check Deposit (Delayed)' else row['transaction_type'], axis=1)


# add transaction day, transaction week, & transaction pay period to transaction table
df_txn['txn_date'] = df_txn.posted_at.apply(lambda x: Timestamp('{}-{}-{}'.format(x.year, x.month, x.day)) if isinstance(x, Timestamp) else None)
df_txn['txn_day'] = df_txn.apply(lambda row: days_since(row['txn_date'], first_day), axis=1)
df_txn['txn_week'] = df_txn.apply(lambda row: weeks_since(row['txn_date'], first_day, monday_start=True), axis=1)
df_txn['txn_month'] = df_txn.posted_at.apply(lambda x: Timestamp('{}-{}-{}'.format(x.year, x.month, x.days_in_month)) if isinstance(x, Timestamp) else None)

txn_period_args = (period, 'txn_week')
txn_period_kwargs = {'weeks' : period_length}
df_txn['txn_period'] = df_txn.apply(append_derived_column, axis=1, args=txn_period_args, **txn_period_kwargs)


# add signup day, signup week, & signup pay period to account table
df_acct['signup_date'] = df_acct.signup_date.apply(lambda x: Timestamp('{}-{}-{}'.format(x.year, x.month, x.day)) if isinstance(x, Timestamp) else None)
df_acct['signup_day'] = df_acct.apply(lambda row: days_since(row['signup_date'], first_day), axis = 1)
df_acct['signup_week'] = df_acct.apply(lambda row: weeks_since(row['signup_date'], first_day, monday_start=True), axis = 1)
df_acct['signup_month'] = df_acct.signup_date.apply(lambda x: Timestamp('{}-{}-{}'.format(x.year, x.month, x.days_in_month)) if isinstance(x, Timestamp) else None)

signup_period_args = (period, 'signup_week')
signup_period_kwargs = {'weeks' : period_length}
df_acct['signup_period'] = df_acct.apply(append_derived_column, axis=1, args=signup_period_args, **signup_period_kwargs)


# add cohort to account table
cohort_args = (cohort, 'signup_week')
cohort_kwargs = {'weeks' : cohort_length}
df_acct['cohort'] = df_acct.apply(append_derived_column, axis=1, args=cohort_args, **cohort_kwargs)

## Create clean transaction, load, and account tables by removing test accounts

In [8]:
# remove test accounts from transaction table
df_txn_clean = df_txn.copy()
df_txn_clean = df_txn_clean[df_txn_clean.account_roles.apply(lambda x: account_type(x)).isin(['Test Account']) == False]
df_txn_clean = df_txn_clean[df_txn_clean.account_roles.apply(lambda x: account_type(x)).isin(['Employee Account']) == False]
df_txn_clean = df_txn_clean[df_txn_clean['account_owner_uid'].isin(hq_test_users) == False]
df_txn_clean = df_txn_clean[df_txn_clean.txn_date <= last_day]


# remove test accounts from account table
df_acct_clean = df_acct.copy()
df_acct_clean = df_acct_clean[df_acct_clean.account_roles.apply(lambda x: account_type(x)).isin(['Test Account']) == False]
df_acct_clean = df_acct_clean[df_acct_clean.account_roles.apply(lambda x: account_type(x)).isin(['Employee Account']) == False]
df_acct_clean = df_acct_clean[df_acct_clean['account_owner_uid'].isin(hq_test_users) == False]
df_acct_clean = df_acct_clean[df_acct_clean.signup_date <= last_day]


# create load only table for further analysis
df_load_clean = df_txn_clean[df_txn_clean.load_ind == True]


# create network debit only table for further analysis
df_net_deb_clean = df_txn_clean[df_txn_clean.transaction_type == 'Network Debit']

## Append conversion date to account table
Note that a load only includes all non-incentive checks, all cash loads, and only ACH transfers $5.00+.

In [9]:
# define first load period per individual
df_first_load = df_load_clean.groupby('account_owner_uid').agg({'txn_period' : 'min', 'txn_day' : 'min', 'txn_month' : 'min'}).reset_index()


# rename columns
df_first_load = df_first_load.rename(columns={'txn_period' : 'conversion_period', 'txn_day' : 'conversion_day', 'txn_month' : 'conversion_month'})


# append first load date to account table
df_acct_clean = df_acct_clean.merge(df_first_load, on='account_owner_uid', how='left')

## Create summary table of cumulative signups and conversions
Note that a load only includes all non-incentive checks, all cash loads, and only ACH transfers $5.00+.

In [10]:
# create table of incremental signups by salesperson by pay period
df_signups_clean = df_acct_clean.groupby(['signup_period']).agg({'account_owner_uid' : 'count'}).cumsum().reset_index()
df_signups_clean = df_signups_clean.rename(columns={'signup_period' : 'period', 'account_owner_uid' : 'signups_sum'})


# create table of incremental conversion by salesperson by pay period
df_conversions_clean = df_acct_clean.groupby(['conversion_period']).agg({'account_owner_uid' : 'count'}).cumsum().reset_index()
df_conversions_clean = df_conversions_clean.rename(columns={'conversion_period' : 'period', 'account_owner_uid' : 'conversions_sum'})


# merge tables to make calculations easier
df_signups_conversions = df_signups_clean.merge(df_conversions_clean, on='period', how='outer').sort('period').fillna(method='ffill')

/Users/kevinkupiec/ipy/lib/python2.7/site-packages/ipykernel/__main__.py:12: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [11]:
# add fields for first and last day of pay period
df_signups_conversions['period_beginning'] = df_signups_conversions.apply(lambda row: first_day + datetime.timedelta(days=((7 * period_length) * row['period'])), axis=1)
df_signups_conversions['period_ending'] = df_signups_conversions.apply(lambda row: first_day + datetime.timedelta(days=((7 * period_length) * row['period']) + (7 * period_length - 1)), axis=1)


# remove pay periods that end past the last day in the dataset or end prior to the first day in the analysis
df_signups_conversions = df_signups_conversions[df_signups_conversions.period_ending <= last_day].fillna(0)
df_signups_conversions = df_signups_conversions[df_signups_conversions.period_ending >= first_day].fillna(0)


# re-order table to make it cleaner later
df_final = df_signups_conversions[['period', 'period_beginning', 'period_ending', 'signups_sum', 'conversions_sum']]

## Create summary table of 45-day load
Note that a load only includes all non-incentive checks, all cash loads, and only ACH transfers $5.00+.

In [12]:
# create summary table that counts loads by day, by individual
df_load_clean_grouped = df_load_clean.groupby(['account_owner_uid', 'txn_day']).agg({'transaction_uid' : 'count'})
df_load_clean_grouped = df_load_clean_grouped.rename(columns={'transaction_uid' : 'daily_loads'}).reset_index()


# create new columns for daily transactions and total transactions with max being 1 as an indicator
df_load_clean_grouped['daily_loads'] = df_load_clean_grouped.apply(lambda row: 1 if 'daily_loads' > 0 else 0, axis=1)


# create table to be used for looping
df_load_clean_loop = df_load_clean_grouped.copy()


# define merge columns for loop
merge_columns = ['account_owner_uid', 'txn_day']


# run loop, which appends prior rolling_days - 1 days based on account owner and day to determine 
# whether load occured in rolling rolling_days day period
for i in range(rolling_days):
    if i == 0:
        continue
    df_load_clean_loop['txn_day'] = df_load_clean_loop['txn_day'] + 1
    df_load_clean_grouped = df_load_clean_grouped.merge(df_load_clean_loop, on=merge_columns, suffixes=('0', i), how='outer')
    df_load_clean_grouped = df_load_clean_grouped.rename(columns={'daily_loads0': 'daily_loads'})

    
# finish calculating rolling view   
df_load_clean_grouped = df_load_clean_grouped.fillna(0)
df_load_clean_grouped = df_load_clean_grouped.set_index(['account_owner_uid', 'txn_day'])
df_load_clean_grouped['rolling_loads'] = df_load_clean_grouped.max(axis=1)
df_load_clean_grouped = df_load_clean_grouped['rolling_loads']
df_load_clean_grouped = df_load_clean_grouped.reset_index()
df_load_clean_rolling = df_load_clean_grouped.groupby(['txn_day']).agg({'rolling_loads' : 'sum'}).reset_index()


# add date to table to aid joining to final table
df_load_clean_rolling['period_ending'] = df_load_clean_rolling.apply(lambda row: first_day + datetime.timedelta(days=row['txn_day']), axis=1)


# reorder table to aid join
df_load_clean_rolling = df_load_clean_rolling[['period_ending', 'rolling_loads']]


# merge with final table
df_final = df_final.merge(df_load_clean_rolling, on='period_ending', how='left')

## Create summary table of 45-day network debit
Note that this is the posted date of the transaction - excludes returns but includes adjustments that are <$0.00.

In [13]:
# create summary table that counts network debits by day, by individual
df_net_deb_clean_grouped = df_net_deb_clean.groupby(['account_owner_uid', 'txn_day']).agg({'transaction_uid' : 'count'})
df_net_deb_clean_grouped = df_net_deb_clean_grouped.rename(columns={'transaction_uid' : 'daily_net_debs'}).reset_index()


# create new columns for daily transactions and total transactions with max being 1 as an indicator
df_net_deb_clean_grouped['daily_net_debs'] = df_net_deb_clean_grouped.apply(lambda row: 1 if 'daily_net_debs' > 0 else 0, axis=1)


# create table to be used for looping
df_net_deb_clean_loop = df_net_deb_clean_grouped.copy()


# define merge columns for loop
merge_columns = ['account_owner_uid', 'txn_day']


# run loop, which appends prior rolling_days - 1 days based on account owner and day to determine 
# whether net_deb occured in rolling rolling_days day period
for i in range(rolling_days):
    if i == 0:
        continue
    df_net_deb_clean_loop['txn_day'] = df_net_deb_clean_loop['txn_day'] + 1
    df_net_deb_clean_grouped = df_net_deb_clean_grouped.merge(df_net_deb_clean_loop, on=merge_columns, suffixes=('0', i), how='outer')
    df_net_deb_clean_grouped = df_net_deb_clean_grouped.rename(columns={'daily_net_debs0': 'daily_net_debs'})

    
# finish calculating rolling view   
df_net_deb_clean_grouped = df_net_deb_clean_grouped.fillna(0)
df_net_deb_clean_grouped = df_net_deb_clean_grouped.set_index(['account_owner_uid', 'txn_day'])
df_net_deb_clean_grouped['rolling_net_debs'] = df_net_deb_clean_grouped.max(axis=1)
df_net_deb_clean_grouped = df_net_deb_clean_grouped['rolling_net_debs']
df_net_deb_clean_grouped = df_net_deb_clean_grouped.reset_index()
df_net_deb_clean_rolling = df_net_deb_clean_grouped.groupby(['txn_day']).agg({'rolling_net_debs' : 'sum'}).reset_index()


# add date to table to aid joining to final table
df_net_deb_clean_rolling['period_ending'] = df_net_deb_clean_rolling.apply(lambda row: first_day + datetime.timedelta(days=row['txn_day']), axis=1)


# reorder table to aid join
df_net_deb_clean_rolling = df_net_deb_clean_rolling[['period_ending', 'rolling_net_debs']]


# merge with final table
df_final = df_final.merge(df_net_deb_clean_rolling, on='period_ending', how='left')

## Create table of count of various transaction types

In [14]:
# create table of transaction sum grouping by debit type
df_txn_clean_sum = df_txn_clean.groupby(['txn_period', 'transaction_type']).agg({'amount' : 'sum'}).unstack().fillna(0)
df_txn_clean_sum.columns = df_txn_clean_sum.columns.droplevel(0)
df_txn_clean_sum['Total Debit'] = df_txn_clean_sum['ATM Withdrawal'] + df_txn_clean_sum['Bill Pay'] + df_txn_clean_sum['Mail a Check'] + df_txn_clean_sum['Network Debit'] + df_txn_clean_sum['Network Credit'] + df_txn_clean_sum['ACH Debit']
df_txn_clean_sum['Total Load'] = df_txn_clean_sum['Check Deposit'] + df_txn_clean_sum['Cash Load'] + df_txn_clean_sum['ACH Credit']
df_txn_clean_sum['Total Transfers'] = df_txn_clean_sum['Transfer from Vault'] + df_txn_clean_sum['Transfer to Vault']


df_sum = df_txn_clean_sum[['Network Debit', 'ATM Withdrawal', 'Total Debit', 
                                           'ACH Credit', 'Cash Load', 'Check Deposit', 'Total Load', 
                                           'Transfer to Vault', 'Transfer from Vault', 'Total Transfers']]
df_sum = df_sum.rename(columns={'Network Debit' : 'Network Debit - Sum', 
                                                'ATM Withdrawal' : 'ATM Withdrawal - Sum', 
                                                'Total Debit' : 'Total Debit - Sum', 
                                                'ACH Credit' : 'ACH Credit - Sum', 
                                                'Cash Load' : 'Cash Load - Sum', 
                                                'Check Deposit' : 'Check Deposit - Sum', 
                                                'Total Load' : 'Total Load - Sum', 
                                                'Transfer to Vault' : 'Transfer to Vault - Sum', 
                                                'Transfer from Vault' : 'Transfer from Vault - Sum', 
                                                'Total Transfers' : 'Total Transfers - Sum'})
df_sum.index.rename('period', inplace=True)
df_sum = df_sum.reset_index()

In [15]:
# create table of transaction count grouping by debit type
df_txn_clean_count = df_txn_clean.groupby(['txn_period', 'transaction_type']).agg({'amount' : 'count'}).unstack().fillna(0)
df_txn_clean_count.columns = df_txn_clean_count.columns.droplevel(0)
df_txn_clean_count['Total Debit'] = df_txn_clean_count['ATM Withdrawal'] + df_txn_clean_count['Bill Pay'] + df_txn_clean_count['Mail a Check'] + df_txn_clean_count['Network Debit'] + df_txn_clean_count['Network Credit'] + df_txn_clean_count['ACH Debit']
df_txn_clean_count['Total Load'] = df_txn_clean_count['Check Deposit'] + df_txn_clean_count['Cash Load'] + df_txn_clean_count['ACH Credit']
df_txn_clean_count['Total Transfers'] = df_txn_clean_count['Transfer from Vault'] + df_txn_clean_count['Transfer to Vault']


df_count = df_txn_clean_count[['Network Debit', 'ATM Withdrawal', 'Total Debit', 
                                           'ACH Credit', 'Cash Load', 'Check Deposit', 'Total Load', 
                                           'Transfer to Vault', 'Transfer from Vault', 'Total Transfers']]
df_count = df_count.rename(columns={'Network Debit' : 'Network Debit - Count', 
                                                'ATM Withdrawal' : 'ATM Withdrawal - Count', 
                                                'Total Debit' : 'Total Debit - Count', 
                                                'ACH Credit' : 'ACH Credit - Count', 
                                                'Cash Load' : 'Cash Load - Count', 
                                                'Check Deposit' : 'Check Deposit - Count', 
                                                'Total Load' : 'Total Load - Count', 
                                                'Transfer to Vault' : 'Transfer to Vault - Count', 
                                                'Transfer from Vault' : 'Transfer from Vault - Count', 
                                                'Total Transfers' : 'Total Transfers - Count'})
df_count.index.rename('period', inplace=True)
df_count = df_count.reset_index()

In [16]:
# merge onto final table
df_final = df_final.merge(df_sum, on='period', how='left').fillna(0)
df_final = df_final.merge(df_count, on='period', how='left').fillna(0)

## Replicate above for a monthly view

In [17]:
# create table of incremental signups by salesperson by pay period
df_signups_month_clean = df_acct_clean.groupby(['signup_month']).agg({'account_owner_uid' : 'count'}).cumsum().reset_index()
df_signups_month_clean = df_signups_month_clean.rename(columns={'signup_month' : 'month', 'account_owner_uid' : 'signups_sum'})


# create table of incremental conversion by salesperson by pay period
df_conversions_month_clean = df_acct_clean.groupby(['conversion_month']).agg({'account_owner_uid' : 'count'}).cumsum().reset_index()
df_conversions_month_clean = df_conversions_month_clean.rename(columns={'conversion_month' : 'month', 'account_owner_uid' : 'conversions_sum'})


# merge tables to make calculations easier
df_signups_conversions_month = df_signups_month_clean.merge(df_conversions_month_clean, on='month', how='outer').sort('month').fillna(method='ffill')


# remove pay periods that end past the last day in the dataset or end prior to the first day in the analysis
df_signups_conversions_month = df_signups_conversions_month[df_signups_conversions_month.month <= last_day].fillna(0)
df_signups_conversions_month = df_signups_conversions_month[df_signups_conversions_month.month >= first_day].fillna(0)


# re-order table to make it cleaner later
df_final_month = df_signups_conversions_month[['month', 'signups_sum', 'conversions_sum']]

/Users/kevinkupiec/ipy/lib/python2.7/site-packages/ipykernel/__main__.py:12: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [18]:
# merge 45 day rolling views
df_final_month = df_final_month.merge(df_load_clean_rolling, left_on='month', right_on='period_ending', how='left')
df_final_month.drop('period_ending', axis=1, inplace=True)
df_final_month = df_final_month.merge(df_net_deb_clean_rolling, left_on='month', right_on='period_ending', how='left')
df_final_month.drop('period_ending', axis=1, inplace=True)

In [19]:
# create table of transaction sum grouping by debit type
df_txn_clean_sum_month = df_txn_clean.groupby(['txn_month', 'transaction_type']).agg({'amount' : 'sum'}).unstack().fillna(0)
df_txn_clean_sum_month.columns = df_txn_clean_sum_month.columns.droplevel(0)
df_txn_clean_sum_month['Total Debit'] = df_txn_clean_sum_month['ATM Withdrawal'] + df_txn_clean_sum_month['Bill Pay'] + df_txn_clean_sum_month['Mail a Check'] + df_txn_clean_sum_month['Network Debit'] + df_txn_clean_sum_month['Network Credit'] + df_txn_clean_sum_month['ACH Debit']
df_txn_clean_sum_month['Total Load'] = df_txn_clean_sum_month['Check Deposit'] + df_txn_clean_sum_month['Cash Load'] + df_txn_clean_sum_month['ACH Credit']
df_txn_clean_sum_month['Total Transfers'] = df_txn_clean_sum_month['Transfer from Vault'] + df_txn_clean_sum_month['Transfer to Vault']


df_sum_month = df_txn_clean_sum_month[['Network Debit', 'ATM Withdrawal', 'Total Debit', 
                                           'ACH Credit', 'Cash Load', 'Check Deposit', 'Total Load', 
                                           'Transfer to Vault', 'Transfer from Vault', 'Total Transfers']]
df_sum_month = df_sum_month.rename(columns={'Network Debit' : 'Network Debit - Sum', 
                                                'ATM Withdrawal' : 'ATM Withdrawal - Sum', 
                                                'Total Debit' : 'Total Debit - Sum', 
                                                'ACH Credit' : 'ACH Credit - Sum', 
                                                'Cash Load' : 'Cash Load - Sum', 
                                                'Check Deposit' : 'Check Deposit - Sum', 
                                                'Total Load' : 'Total Load - Sum', 
                                                'Transfer to Vault' : 'Transfer to Vault - Sum', 
                                                'Transfer from Vault' : 'Transfer from Vault - Sum', 
                                                'Total Transfers' : 'Total Transfers - Sum'})
df_sum_month.index.rename('month', inplace=True)
df_sum_month = df_sum_month.reset_index()


# create table of transaction count grouping by debit type
df_txn_clean_count_month = df_txn_clean.groupby(['txn_month', 'transaction_type']).agg({'amount' : 'count'}).unstack().fillna(0)
df_txn_clean_count_month.columns = df_txn_clean_count_month.columns.droplevel(0)
df_txn_clean_count_month['Total Debit'] = df_txn_clean_count_month['ATM Withdrawal'] + df_txn_clean_count_month['Bill Pay'] + df_txn_clean_count_month['Mail a Check'] + df_txn_clean_count_month['Network Debit'] + df_txn_clean_count_month['Network Credit'] + df_txn_clean_count_month['ACH Debit']
df_txn_clean_count_month['Total Load'] = df_txn_clean_count_month['Check Deposit'] + df_txn_clean_count_month['Cash Load'] + df_txn_clean_count_month['ACH Credit']
df_txn_clean_count_month['Total Transfers'] = df_txn_clean_count_month['Transfer from Vault'] + df_txn_clean_count_month['Transfer to Vault']


df_count_month = df_txn_clean_count_month[['Network Debit', 'ATM Withdrawal', 'Total Debit', 
                                           'ACH Credit', 'Cash Load', 'Check Deposit', 'Total Load', 
                                           'Transfer to Vault', 'Transfer from Vault', 'Total Transfers']]
df_count_month = df_count_month.rename(columns={'Network Debit' : 'Network Debit - Count', 
                                                'ATM Withdrawal' : 'ATM Withdrawal - Count', 
                                                'Total Debit' : 'Total Debit - Count', 
                                                'ACH Credit' : 'ACH Credit - Count', 
                                                'Cash Load' : 'Cash Load - Count', 
                                                'Check Deposit' : 'Check Deposit - Count', 
                                                'Total Load' : 'Total Load - Count', 
                                                'Transfer to Vault' : 'Transfer to Vault - Count', 
                                                'Transfer from Vault' : 'Transfer from Vault - Count', 
                                                'Total Transfers' : 'Total Transfers - Count'})
df_count_month.index.rename('month', inplace=True)
df_count_month = df_count_month.reset_index()

In [20]:
# merge onto final table
df_final_month = df_final_month.merge(df_sum_month, on='month', how='left').fillna(0)
df_final_month = df_final_month.merge(df_count_month, on='month', how='left').fillna(0)

## Create table of conversion rates by cohort
Conversion rates are as of 2 weeks, 4 weeks, 6 weeks, & 8 weeks. Note that the first day counted is the day the individual signed up.

In [21]:
# calculate number of days from signup until conversion
df_acct_clean['days_to_convert'] = df_acct_clean['conversion_day'] - df_acct_clean['signup_day'] + 1


# create four columns that flag whether the individual converted in 2, 4, 6, & 8 weeks and total signups in cohort
df_acct_clean['_14'] = df_acct_clean.apply(lambda row: 1 if row['days_to_convert'] <= 14 else 0, axis=1)
df_acct_clean['_28'] = df_acct_clean.apply(lambda row: 1 if row['days_to_convert'] <= 28 else 0, axis=1)
df_acct_clean['_42'] = df_acct_clean.apply(lambda row: 1 if row['days_to_convert'] <= 42 else 0, axis=1)
df_acct_clean['_56'] = df_acct_clean.apply(lambda row: 1 if row['days_to_convert'] <= 56 else 0, axis=1)
df_acct_clean['_cumulative'] = df_acct_clean.apply(lambda row: 1 if row['days_to_convert'] >= 0 else 0, axis=1)
df_cohort_clean = df_acct_clean.groupby('cohort').agg({'_14' : 'sum', '_28' : 'sum', '_42' : 'sum', '_56' : 'sum', '_cumulative' : 'sum', 'account_owner_uid' : 'count'}).reset_index().fillna(0)


# add fields for first and last day of cohort
df_cohort_clean['cohort_beginning'] = df_cohort_clean.apply(lambda row: first_day + datetime.timedelta(days=((7 * cohort_length) * row['cohort'])), axis=1)
df_cohort_clean['cohort_ending'] = df_cohort_clean.apply(lambda row: first_day + datetime.timedelta(days=((7 * cohort_length) * row['cohort']) + (7 * cohort_length - 1)), axis=1)


# null out fields that haven't had enough days to age
df_cohort_clean['_14'] = df_cohort_clean.apply(lambda row: float(row['_14']) if (row['cohort_ending'] + datetime.timedelta(days=13)) <= last_day else None, axis=1)
df_cohort_clean['_28'] = df_cohort_clean.apply(lambda row: float(row['_28']) if (row['cohort_ending'] + datetime.timedelta(days=27)) <= last_day else None, axis=1)
df_cohort_clean['_42'] = df_cohort_clean.apply(lambda row: float(row['_42']) if (row['cohort_ending'] + datetime.timedelta(days=41)) <= last_day else None, axis=1)
df_cohort_clean['_56'] = df_cohort_clean.apply(lambda row: float(row['_56']) if (row['cohort_ending'] + datetime.timedelta(days=55)) <= last_day else None, axis=1)


# create conversion percentages
df_cohort_clean['_14_pct'] = df_cohort_clean['_14'] / df_cohort_clean['account_owner_uid']
df_cohort_clean['_28_pct'] = df_cohort_clean['_28'] / df_cohort_clean['account_owner_uid']
df_cohort_clean['_42_pct'] = df_cohort_clean['_42'] / df_cohort_clean['account_owner_uid']
df_cohort_clean['_56_pct'] = df_cohort_clean['_56'] / df_cohort_clean['account_owner_uid']
df_cohort_clean['_cumulative_pct'] = df_cohort_clean['_cumulative'] / df_cohort_clean['account_owner_uid']


# remove cohorts that end past the last day in the dataset or end prior to the first day in the analysis
df_cohort_clean = df_cohort_clean[df_cohort_clean.cohort_ending <= last_day]
df_cohort_clean = df_cohort_clean[df_cohort_clean.cohort_ending >= first_day]


# create final table
df_cohort = df_cohort_clean[['cohort', 'cohort_beginning', 'cohort_ending', 'account_owner_uid', '_14_pct', '_28_pct', '_42_pct', '_56_pct', '_cumulative_pct']]

## Export tables to Excel file

In [26]:
# update column names
df_final = df_final.rename(columns={'period' : 'Period', 'period_beginning' : 'Period Start Date', 'period_ending' : 'Period End Date', 'signups_sum' : 'Cumulative Signups', 'conversions_sum' : 'Conversions Sum', 'rolling_loads' : 'Rolling Loads', 'rolling_net_debs' : 'Rolling Network Debits'})
df_final_month = df_final_month.rename(columns={'month' : 'Month', 'signups_sum' : 'Cumulative Signups', 'conversions_sum' : 'Conversions Sum', 'rolling_loads' : 'Rolling Loads', 'rolling_net_debs' : 'Rolling Network Debits'})
df_cohort = df_cohort.rename(columns={'cohort' : 'Cohort', 'cohort_beginning' : 'Cohort Start Date', 'cohort_ending' : 'Cohort End Date', 'account_owner_uid' : 'Signups', '_14_pct' : 'Two Week Conversion Rate', '_28_pct' : 'Four Week Conversion Rate', '_42_pct' : 'Six Week Conversion Rate', '_56_pct' : 'Eight Week Conversion Rate', '_cumulative_pct' : 'Cumulative Conversion Rate'})


# sort tables
df_final = df_final.sort('Period')
df_final_month = df_final_month.sort('Month')
df_cohort = df_cohort.sort('Cohort')


# define file name
file_name = file_path + last_day.strftime('%Y%m%d') + ' Weekly Reporting.xlsx'


# write to file
writer = pd.ExcelWriter(file_name)
df_final.to_excel(writer, sheet_name='Data by Period')
df_final_month.to_excel(writer, sheet_name='Data by Month')
df_cohort.to_excel(writer, sheet_name='Cohort Conversion Rates')
df_ach_cohort.to_excel(writer, sheet_name='Cohort ACH Conversion Rates')
writer.save()

/Users/kevinkupiec/ipy/lib/python2.7/site-packages/ipykernel/__main__.py:8: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/Users/kevinkupiec/ipy/lib/python2.7/site-packages/ipykernel/__main__.py:9: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/Users/kevinkupiec/ipy/lib/python2.7/site-packages/ipykernel/__main__.py:10: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [23]:
# helper function to identify loads that count towards employee
def ach_load(transaction_type, amount):
    if transaction_type == 'ACH Credit' and amount >= 5:
        return True
    else:
        return False
    
    
# append field that identifies loads
ach_load_args = (ach_load, 'transaction_type', 'amount')
df_txn['ach_load_ind'] = df_txn.apply(append_derived_column, axis=1, args=ach_load_args)

In [24]:
df_acct_clean = df_acct.copy()
df_acct_clean = df_acct_clean[df_acct_clean.account_roles.apply(lambda x: account_type(x)).isin(['Test Account']) == False]
df_acct_clean = df_acct_clean[df_acct_clean.account_roles.apply(lambda x: account_type(x)).isin(['Employee Account']) == False]
df_acct_clean = df_acct_clean[df_acct_clean['account_owner_uid'].isin(hq_test_users) == False]
df_acct_clean = df_acct_clean[df_acct_clean.signup_date <= last_day]

In [25]:
# create load only table for further analysis
df_ach_load_clean = df_txn[df_txn.ach_load_ind == True]


# define first load period per individual
df_first_ach_load = df_ach_load_clean.groupby('account_owner_uid').agg({'txn_period' : 'min', 'txn_day' : 'min', 'txn_month' : 'min'}).reset_index()


# rename columns
df_first_ach_load = df_first_ach_load.rename(columns={'txn_period' : 'ach_conversion_period', 'txn_day' : 'ach_conversion_day', 'txn_month' : 'ach_conversion_month'})


# append first load date to account table
df_acct_clean = df_acct_clean.merge(df_first_ach_load, on='account_owner_uid', how='left')


# calculate number of days from signup until conversion
df_acct_clean['ach_days_to_convert'] = df_acct_clean['ach_conversion_day'] - df_acct_clean['signup_day'] + 1


# create four columns that flag whether the individual converted in 2, 4, 6, & 8 weeks and total signups in cohort
df_acct_clean['ach_14'] = df_acct_clean.apply(lambda row: 1 if row['ach_days_to_convert'] <= 14 else 0, axis=1)
df_acct_clean['ach_28'] = df_acct_clean.apply(lambda row: 1 if row['ach_days_to_convert'] <= 28 else 0, axis=1)
df_acct_clean['ach_42'] = df_acct_clean.apply(lambda row: 1 if row['ach_days_to_convert'] <= 42 else 0, axis=1)
df_acct_clean['ach_56'] = df_acct_clean.apply(lambda row: 1 if row['ach_days_to_convert'] <= 56 else 0, axis=1)
df_acct_clean['ach_cumulative'] = df_acct_clean.apply(lambda row: 1 if row['ach_days_to_convert'] >= 0 else 0, axis=1)
df_ach_cohort_clean = df_acct_clean.groupby('cohort').agg({'ach_14' : 'sum', 'ach_28' : 'sum', 'ach_42' : 'sum', 'ach_56' : 'sum', 'ach_cumulative' : 'sum', 'account_owner_uid' : 'count'}).reset_index().fillna(0)


# add fields for first and last day of cohort
df_ach_cohort_clean['cohort_beginning'] = df_ach_cohort_clean.apply(lambda row: first_day + datetime.timedelta(days=((7 * cohort_length) * row['cohort'])), axis=1)
df_ach_cohort_clean['cohort_ending'] = df_ach_cohort_clean.apply(lambda row: first_day + datetime.timedelta(days=((7 * cohort_length) * row['cohort']) + (7 * cohort_length - 1)), axis=1)


# null out fields that haven't had enough days to age
df_ach_cohort_clean['ach_14'] = df_ach_cohort_clean.apply(lambda row: float(row['ach_14']) if (row['cohort_ending'] + datetime.timedelta(days=13)) <= last_day else None, axis=1)
df_ach_cohort_clean['ach_28'] = df_ach_cohort_clean.apply(lambda row: float(row['ach_28']) if (row['cohort_ending'] + datetime.timedelta(days=27)) <= last_day else None, axis=1)
df_ach_cohort_clean['ach_42'] = df_ach_cohort_clean.apply(lambda row: float(row['ach_42']) if (row['cohort_ending'] + datetime.timedelta(days=41)) <= last_day else None, axis=1)
df_ach_cohort_clean['ach_56'] = df_ach_cohort_clean.apply(lambda row: float(row['ach_56']) if (row['cohort_ending'] + datetime.timedelta(days=55)) <= last_day else None, axis=1)


# create conversion percentages
df_ach_cohort_clean['_14_pct'] = df_ach_cohort_clean['ach_14'] / df_ach_cohort_clean['account_owner_uid']
df_ach_cohort_clean['_28_pct'] = df_ach_cohort_clean['ach_28'] / df_ach_cohort_clean['account_owner_uid']
df_ach_cohort_clean['_42_pct'] = df_ach_cohort_clean['ach_42'] / df_ach_cohort_clean['account_owner_uid']
df_ach_cohort_clean['_56_pct'] = df_ach_cohort_clean['ach_56'] / df_ach_cohort_clean['account_owner_uid']
df_ach_cohort_clean['_cumulative_pct'] = df_ach_cohort_clean['ach_cumulative'] / df_ach_cohort_clean['account_owner_uid']


# remove cohorts that end past the last day in the dataset or end prior to the first day in the analysis
df_ach_cohort_clean = df_ach_cohort_clean[df_ach_cohort_clean.cohort_ending <= last_day]
df_ach_cohort_clean = df_ach_cohort_clean[df_ach_cohort_clean.cohort_ending >= first_day]


# create final table
df_ach_cohort = df_ach_cohort_clean[['cohort', 'cohort_beginning', 'cohort_ending', 'account_owner_uid', '_14_pct', '_28_pct', '_42_pct', '_56_pct', '_cumulative_pct']]